In [ ]:
#본 내용은 kaggle Intermediate Machine Learning course(by Alexis Cook)를 기록해 놓은 것임을 밝힙니다. (https://www.kaggle.com/learn/intermediate-machine-learning)

In [1]:
from google.colab import drive 
drive.mount('/content/drive')
#구글 코랩환경 드라이브 연결

Mounted at /content/drive


**categorical variable를 ML 모델에 연결시킬 때 필요한 세가지 준비 방법**

1) Drop Categorical Variables 

간단하게 Drop해주는 방법. 당연히 열에 유용한 정보가 포함되어 있지 않을때만 사용 가능합니다.

2) Label Encoding

각 고유값을 다른 정수에 할당하는 방법. 범주의 순서를 고려한다. decision trees and random forests와 같은 트리 기반 모델 에서 다른 순서가 있는 변수와 잘 작동할 수 있습니다.

3) One-Hot Encoding

이 변수가 있는지(1) 없는지(0)를 나타내는 새로운 열을 생성하는 방법. 범주의 순서를 고려하지 않습니다. 범주형 데이터에 명확한 순서관계가 드러나지 않은경우 잘 작동할 수 있습니다. 물론 열을 새로 만들기 때문에 범주형 변수가 많은 값(15이상)을 차지하면 제대로 수행되지 않습니다.








**Example**

In [8]:
import pandas as pd
from sklearn.model_selection import train_test_split
#데이터 불러오기
data=pd.read_csv('/content/drive/MyDrive/ML-Study-Jam-in-DSC-Sookmyung/Intermediate Machine Learning/data/melb_data.csv')

y=data.Price
X=data.drop(['Price'],axis=1)
#training and validation subsets으로 나누기
X_train_full, X_valid_full, y_train, y_valid=train_test_split(X,y,train_size=0.8,test_size=0.2,random_state=0)
#결측치가 있는 열을 drop합니다.
cols_with_missing =[col for col in X_train_full.columns if X_train_full[col].isnull().any()]
X_train_full.drop(cols_with_missing, axis=1, inplace=True)
X_valid_full.drop(cols_with_missing, axis=1, inplace=True)
#상대적으로 low cardinality을 가진 categorical columns을 선택합니다.(임의로 선택한 것으로 편리하긴 합니다.)
low_cardinality_cols=[cname for cname in X_train_full.columns if X_train_full[cname].nunique()<10 and X_train_full[cname].dtype=='object']
#숫자타입의 열을 선택합니다.
numerical_cols=[cname for cname in X_train_full.columns if X_train_full[cname].dtype in ['int64','float64']]

my_cols=low_cardinality_cols + numerical_cols
X_train=X_train_full[my_cols].copy()
X_valid=X_valid_full[my_cols].copy()

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [9]:
X_train.head()

,Type,Method,Regionname,Rooms,Distance,Postcode,Bedroom2,Bathroom,Landsize,Lattitude,Longtitude,Propertycount
12167,u,S,Southern Metropolitan,1,5.0,3182.0,1.0,1.0,0.0,-37.85984,144.9867,13240.0
6524,h,SA,Western Metropolitan,2,8.0,3016.0,2.0,2.0,193.0,-37.85800,144.9005,6380.0
8413,h,S,Western Metropolitan,3,12.6,3020.0,3.0,1.0,555.0,-37.79880,144.8220,3755.0
2919,u,SP,Northern Metropolitan,3,13.0,3046.0,3.0,1.0,265.0,-37.70830,144.9158,8870.0
6043,h,S,Western Metropolitan,3,13.3,3020.0,3.0,1.0,673.0,-37.76230,144.8272,4217.0


In [10]:
#categorical variables의 리스트를 가져옵니다
s=(X_train.dtypes=='object')
object_cols = list(s[s].index)

print("Categorical variables:")
print(object_cols)

Categorical variables:
['Type', 'Method', 'Regionname']


In [13]:
#정확도 판단을 위해 MAE기반 score함수를 작성하는 부분
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

def score_dataset(X_train, X_valid, y_train, y_valid):
  model=RandomForestRegressor(n_estimators=100, random_state=0)
  model.fit(X_train, y_train)
  preds=model.predict(X_valid)
  return mean_absolute_error(y_valid,preds)

In [14]:
#Drop Categorical Variables 방법에서 정확도
drop_X_train=X_train.select_dtypes(exclude=['object'])
drop_X_valid=X_valid.select_dtypes(exclude=['object'])

print("MAE from Approach 1 (Drop categorical variables):")
print(score_dataset(drop_X_train, drop_X_valid, y_train, y_valid))

MAE from Approach 1 (Drop categorical variables):
175703.48185157913


In [15]:
#Label Encoding 방법에서 정확도
from sklearn.preprocessing import LabelEncoder

label_X_train=X_train.copy()
label_X_valid=X_valid.copy()

label_encoder=LabelEncoder()
for col in object_cols:
  label_X_train[col]=label_encoder.fit_transform(X_train[col])
  label_X_valid[col]=label_encoder.transform(X_valid[col])

print("MAE from Approach 2 (Label Encoding):")
print(score_dataset(label_X_train, label_X_valid, y_train, y_valid))
#임의로 정수를 할당했기 때문에, 사용자가 정의한 label 일때 더 나은 성능을 기대할 수 있습니다.

MAE from Approach 2 (Label Encoding):
165936.40548390493


In [17]:
#One-Hot Encoding 방법에서 정확도
from sklearn.preprocessing import OneHotEncoder

OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
OH_cols_train = pd.DataFrame(OH_encoder.fit_transform(X_train[object_cols]))
OH_cols_valid = pd.DataFrame(OH_encoder.transform(X_valid[object_cols]))

OH_cols_train.index = X_train.index
OH_cols_valid.index = X_valid.index

num_X_train = X_train.drop(object_cols, axis=1)
num_X_valid = X_valid.drop(object_cols, axis=1)

OH_X_train = pd.concat([num_X_train, OH_cols_train], axis=1)
OH_X_valid = pd.concat([num_X_valid, OH_cols_valid], axis=1)

print("MAE from Approach 3 (One-Hot Encoding):") 
print(score_dataset(OH_X_train, OH_X_valid, y_train, y_valid))

MAE from Approach 3 (One-Hot Encoding):
166089.4893009678
